!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!This script contains maskign code before standardisation. however standardisation of inputs and targets was performed before the interpoléated baselines were generated via CDO for bilinear and bicubic and then masked so that they only retain the Swiss Domain

Workflow : Bilinear interpolation, bicubic interpolation, Masking to retain only the swiss domain, splitting into train, test val, gridded RMSE R2, pooled RMSE, R2

In [1]:
from dependencies import *
import sys
sys.path.append("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/Scripts/Functions")

In [2]:
from dask.distributed import Client
client= Client()
print(client)

<Client: 'tcp://127.0.0.1:32941' processes=1 threads=1, memory=503.49 GiB>


2025-05-01 16:15:04,727 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:44389' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {('open_dataset-pr-97d0eba163aba6bc1367edc4fc42ac4f', 12, 0, 0), ('shuffle-split-1a776084eb444a52be381d594c9cc658', 82), ('open_dataset-pr-97d0eba163aba6bc1367edc4fc42ac4f', 10, 0, 0), 'shuffle-taker-9492e4b78523c7fc04aa4e2c834e44bf', ('open_dataset-pr-97d0eba163aba6bc1367edc4fc42ac4f', 21, 0, 0), ('shuffle-split-1a776084eb444a52be381d594c9cc658', 85), ('shuffle-split-1a776084eb444a52be381d594c9cc658', 88), ('open_dataset-pr-97d0eba163aba6bc1367edc4fc42ac4f', 9, 0, 0), ('open_dataset-pr-97d0eba163aba6bc1367edc4fc42ac4f', 18, 0, 0), ('open_dataset-pr-97d0eba163aba6bc1367edc4fc42ac4f', 6, 0, 0), ('shuffle-split-1a776084eb444a52be381d594c9cc658', 75), ('shuffle-split-1a776084eb444a52be381d594c9cc658', 78), ('shuffle-split-1a776084eb444a52be381d594c9cc658', 84), ('shuffle-split-1a776084eb444a52be381d594c

bilinear interpolation to the 1 km grid was performed using CDO remapbil with RhiresD/TabsD as reference files. Same for remapbic (Bicubic interpolation) Now they will be masked in this script so that it only retains the Swiss domain and not entire Europe

#For precipitation mask

In [3]:
hr_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/raw/RhiresD_1971_2022.nc"
mask_output_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/RhiresD_mask.nc"

ds_hr = xr.open_dataset(hr_path)

varname = "RhiresD" 

mask_rhiresd = xr.where(~np.isnan(ds_hr[varname].isel(time=0)), 1, 0)

mask_rhiresd = mask_rhiresd.squeeze(drop=True)

mask_rhiresd.to_netcdf(mask_output_path)

print(f"Mask saved at {mask_output_path}")


Mask saved at /work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/RhiresD_mask.nc


#For temperature mask!

In [6]:
hr_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/raw/TabsD_1971_2022.nc"
mask_output_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/TabsD_mask.nc"

ds_hr = xr.open_dataset(hr_path)

varname = "TabsD" 

mask_tabsd = xr.where(~np.isnan(ds_hr[varname].isel(time=0)), 1, 0)

mask_tabsd = mask_tabsd.squeeze(drop=True)



In [7]:
mask_tabsd.to_netcdf(mask_output_path)

print(f"Mask saved at {mask_output_path}")

Mask saved at /work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/TabsD_mask.nc


#Application of the mask to the two datasets : RhiresD and TabsD masks (1km) to the bicubically interpolated 1 km datasets features from Sven`s files so that it only retains the Swiss Domain


Lazy loading was used below to allow for parallel computation. The data was not loading due to xarray loading everything at once and the kernel was crashing

In [11]:
bicubic_ds_precip = xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processing/features_precip_bicubic.nc",chunks={"time": 500})

# Masking the features to retain the swiss domain
masked_bicubic_rhiresd = bicubic_ds_precip * mask_rhiresd

masked_bicubic_rhiresd.to_netcdf("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/features_precip_masked_bicubic.nc",
                                 engine="netcdf4",
                                    compute=True)

In [12]:
bicubic_ds_temp = xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processing/features_tas_bicubic.nc",chunks={"time": 500})

# Masking the features to retain the swiss domain
masked_bicubic_temp = bicubic_ds_temp * mask_tabsd

masked_bicubic_temp.to_netcdf("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/features_tas_masked_bicubic.nc",
                                 engine="netcdf4",
                                    compute=True)

For reducing computation time
#Could have used h5netcdf for faster computation
#Could have asked for more resources installed dask distributed, and used all 4 CPU cores, computing time reduced to 30 seconds!!!

Bilinear interpolated outputs : Maskign with the 1 km RhiresD and TabsD mask

In [13]:
bilinear_ds_precip = xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processing/features_precip_biliear.nc",chunks={"time": 500})

# Masking
masked_bilinear_precip = bilinear_ds_precip * mask_rhiresd

masked_bilinear_precip.to_netcdf("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bilinear/features_precip_masked_bilinear.nc",
                                 engine="netcdf4",
                                    compute=True)

In [14]:
bilinear_ds_temp = xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processing/features_tas_biliear.nc",chunks={"time": 500})

# Masking
masked_bilinear_temp = bilinear_ds_temp * mask_tabsd

masked_bilinear_temp.to_netcdf("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bilinear/features_tas_masked_bilinear.nc",
                                 engine="netcdf4",
                                    compute=True)

xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxCHECKING SHAPExxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

In [15]:

features_ds_precip = xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/features_precip_masked_bicubic.nc")["pr"]

In [16]:
features_ds_temp= xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/features_tas_masked_bicubic.nc")["tas"]

In [17]:
features_ds_precip.shape

(18993, 265, 370)

In [18]:
features_ds_temp.shape

(18993, 265, 370)

In [19]:
targets_precip= xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/raw/RhiresD_1971_2022.nc")["RhiresD"]
targets_precip.shape

(18993, 265, 370)

In [20]:
targets_temp= xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/raw/TabsD_1971_2022.nc")["TabsD"]
targets_temp.shape

(18993, 265, 370)

SHAPES MATCH!!!!!!

The entire dataset will be divided into tain, test val for both bilinear and bicubic interpolation for calculating standardisation metric etc


Split will be performed on one dataset, then replicated for the other


In [28]:
import sys
sys.path.append("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/ML_models")

from Train_Test_Val import split_by_decade


In [29]:
precip_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/features_precip_masked_bicubic.nc"
tas_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/features_tas_masked_bicubic.nc"

output_base = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic"
ds_precip = xr.open_dataset(precip_path, chunks={"time": 50})
ds_tas = xr.open_dataset(tas_path, chunks={"time": 50})

times = ds_precip['time'].values  # Assumes both datasets have matching time axis, one of the time coordinates from both datasets extracted


In [30]:
train_idx, val_idx, test_idx = split_by_decade(times, seed=42)  # See has been fixed for reproducibility

In [31]:
import os

In [32]:
train_idx, val_idx, test_idx

(tensor([ 2751,  2462,  2313,  ..., 18657, 18129, 18958]),
 tensor([  486,  1772,  1052,  ..., 18762, 18652, 18403]),
 tensor([ 2936,   400,  3011,  ..., 18911, 18736, 18547]))

In [33]:
print(train_idx.shape)
print(val_idx.shape)
print(test_idx.shape)

torch.Size([13293])
torch.Size([3796])
torch.Size([1904])


In [34]:
def save_splits(ds, indices, subset_folder, filename):
    subset = ds.isel(time=indices)
    subset = subset.chunk({"time": 50}) 
    save_dir = os.path.join(output_base, subset_folder)
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, filename)
    
    delayed_obj = subset.to_netcdf(save_path, engine="netcdf4", compute=True)
    delayed_obj.compute()
    
    print(f"Saved {filename}")


In [ ]:
# Precipitation input dataset for bicubic interpolation split into three categories
save_splits(ds_precip, train_idx, "Train", "features_precip_masked_bicubic_train.nc")
save_splits(ds_precip, val_idx, "Val", "features_precip_masked_bicubic_val.nc")
save_splits(ds_precip, test_idx, "Test", "features_precip_masked_bicubic_test.nc")

In [ ]:
# Temperature input dataset for bicubic interpolation split into three categories
save_splits(ds_tas, train_idx, "Train", "features_tas_masked_bicubic_train.nc")
save_splits(ds_tas, val_idx, "Val", "features_tas_masked_bicubic_val.nc")
save_splits(ds_tas, test_idx, "Test", "features_tas_masked_bicubic_test.nc")

Also splittign the targets along the very same lines usign the reproducible split

In [ ]:
precip_target = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/raw/RhiresD_1971_2022.nc"
tas_target = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/raw/TabsD_1971_2022.nc"

output_base = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic"
ds_precip = xr.open_dataset(precip_path, chunks={"time": 50})
ds_tas = xr.open_dataset(tas_path, chunks={"time": 50})

times = ds_precip['time'].values  # Assumes both datasets have matching time axis, one of the time coordinates from both datasets extracted


In [ ]:
# Precipitation input dataset for bicubic interpolation split into three categories
save_splits(precip_target, train_idx, "Train", "targets_precip_masked_bicubic_train.nc")
save_splits(precip_target, val_idx, "Val", "targets_precip_masked_bicubic_val.nc")
save_splits(precip_target, test_idx, "Test", "targets_precip_masked_bicubic_test.nc")

In [ ]:
# Precipitation input dataset for bicubic interpolation split into three categories
save_splits(tas_target, train_idx, "Train", "targets_tas_masked_bicubic_train.nc")
save_splits(tas_target, val_idx, "Val", "targets_tas_masked_bicubic_val.nc")
save_splits(tas_target, test_idx, "Test", "targets_tas_masked_bicubic_test.nc")

                                                  xxxxxxxxxxStandardisation code  xxxxxxxxxxxxxxxxxxxxxxxxxx

In [5]:
from Standardise import standardise

In [5]:
input_path = '/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/raw/TabsD_1971_2022.nc'
output_path = '/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/TabsD_scaled_1971_2022.nc'
var = 'TabsD'

standardise(input_path, output_path, var)


/users/sasthana/.local/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:2053: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


Normalised/min max scaled variable saved to /work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/TabsD_scaled_1971_2022.nc


In [6]:
input_path = '/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/raw/RhiresD_1971_2022.nc'
output_path = '/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/RhiresD_scaled_1971_2022.nc'
var = 'RhiresD'

standardise(input_path, output_path, var)

Normalised/min max scaled variable saved to /work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/RhiresD_scaled_1971_2022.nc


In [3]:
#standardiigng features as well

In [6]:
input_path = '/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/raw/tas_daily_1971_2022_cropped.nc'
output_path = '/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/tas_daily_cropped_scaled_1971_2022.nc'
var = 'tas'

standardise(input_path, output_path, var)

/users/sasthana/.local/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:2053: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


Normalised/min max scaled variable saved to /work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/tas_daily_cropped_scaled_1971_2022.nc


In [7]:
input_path = '/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/raw/precip_daily_1971_2022_cropped.nc'
output_path = '/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/precip_daily_cropped_scaled_1971_2022.nc'
var = 'pr'

standardise(input_path, output_path, var)

Normalised/min max scaled variable saved to /work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/precip_daily_cropped_scaled_1971_2022.nc
